Part 3 of week 3 assignment

In [33]:
from bs4 import BeautifulSoup 
import requests  

import numpy as np 

import lxml
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install geopy
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import json 

import requests 
from pandas.io.json import json_normalize 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


Get the data from wikipedia


In [2]:

url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")

In [3]:
table = soup.find_all("table",{"class":"wikitable sortable"})
table = table[1]

In [4]:
df = pd.read_html(str(table), flavor='html5lib')
df = df[0]
df.head()

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN


In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
def get_coordinates(neighbourhood, borough):
    address = str(neighbourhood)+", "+borough+',Toronto, Ontario, Canada'
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        postcode = str(location).split(',')[-2]
        postcode = postcode.split(" ")[1]
        return [postcode, latitude, longitude]
    else:
        neighbourhoods = neighbourhood.split('-')
        for neighbourhood in neighbourhoods:
            address = str(neighbourhood)+","+borough+", Toronto, Ontario, Canada"
            location = geolocator.geocode(address)
            if location != None:
                latitude = location.latitude
                longitude = location.longitude
                postcode = str(location).split(',')[-2]
                postcode = postcode.split(" ")[1]
                return [postcode, latitude, longitude]
        address = borough+", Toronto, Ontario, Canada"
        location = geolocator.geocode(address)
        if location != None:
                latitude = location.latitude
                longitude = location.longitude
                postcode = str(location).split(',')[-2]
                postcode = postcode.split(" ")[1]
                return [postcode, latitude, longitude]
        return [None, None, None]

In [8]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for row in df.values:
    borough = str(row[2])
    neighbourhood = str(row[1])
    [postcode, latitude, longitude] = get_coordinates(neighbourhood, borough)
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                                            'Borough': borough,
                                            'Neighborhood': neighbourhood,
                                            'Latitude': latitude,
                                            'Longitude': longitude}, ignore_index=True)

In [10]:
neighborhoods.head(140)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1V,Scarborough,Agincourt North,43.808038,-79.266439
1,M1S,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689
2,M8W,Etobicoke,Alderwood,43.601717,-79.545232
3,M5T,Old City of Toronto,Annex,43.670338,-79.407117
4,M2L,North York,Banbury-Don Mills,43.752339,-79.365716
5,Ontario,North York,Bathurst Manor,43.763893,-79.456367
6,M5R,Old City of Toronto,Bay Street Corridor,43.672135,-79.390499
7,M2K,North York,Bayview Village,43.769197,-79.376662
8,M2M,North York,Bayview Woods-Steeles,43.798127,-79.382973
9,M5M,North York,Bedford Park-Nortown,43.737388,-79.410925


Create a map of Toronto

In [11]:
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [38]:
CLIENT_ID = '08111299' # Foursquare ID
CLIENT_SECRET = '08111299' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 08111299
CLIENT_SECRET:08111299


In [45]:
 def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Agincourt North


KeyError: 'groups'